# Amazon SageMaker Neo でコンパイルしたモデルを AWS IoT Greengrass V2 を使ってデバイスにデプロイする

このサンプルノートブックは、エッジ推論を行うために学習済みモデルを Amazon SageMaker Neo でコンパイルして AWS Iot Greengrass V2 を使ってデバイスにデプロイするパイプラインを AWS Step Functions を使って自動化する方法をご紹介します。このノートブックを Amazon SageMaker のノートブックインスタンスで使用する場合は、`conda_tensorflow_p36` のカーネルをご利用ください。

このノートブックでは、デプロイしたいモデルや Greengrass アーティファクトファイルに関する情報を yaml 形式の設定ファイルで作成し、ワークフロー実行時にその設定ファイルを入力パラメタとすることで、以下のことを実現しています。

- いつ何をどのデバイスにデプロイしたのかの記録（トレーサビリティ）
- 同じ設定ファイルを使用することで同じワークフローを実行可能（再現性）

Python コードでワークフローを構築するために、AWS Step Functions Data Science SDK を使用します。詳しい情報は以下のドキュメントをご参照ください。

- [AWS Step Functions](https://aws.amazon.com/step-functions/)
- [AWS Step Functions Developer Guide](https://docs.aws.amazon.com/step-functions/latest/dg/welcome.html)
- [AWS Step Functions Data Science SDK](https://aws-step-functions-data-science-sdk.readthedocs.io/)

このノートブックの大まかな流れは以下の通りです。

![flow](flow.png)

1. 3つの Lambda 関数を作成
  - デプロイしたいコンポーネントに対応するコンテナイメージをデプロイ用 Lambda 関数に適用する Lambda 関数
  - 必要に応じて機械学習モデルを Amazon SageMaker Neo でコンパイルする Lambda 関数
  - 指定されたアーティファクトをコンポーネント化して AWS IoT Greengrass デバイスにデプロイする Lambda 関数
1. 作成した Lambda 関数を順に実行するような AWS Step Functions Data Science SDK ワークフローを作成
1. デプロイに関する情報が記載された設定ファイルを作成
1. Step Functions ワークフローを実行してファイルをデバイスにデプロイ
1. デプロイ関連情報を一覧表示
1. リソースの削除



<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#[事前準備]-AWS-IoT-サービスとデバイス（AWS-Cloud9）のセットアップ" data-toc-modified-id="[事前準備]-AWS-IoT-サービスとデバイス（AWS-Cloud9）のセットアップ-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>[事前準備] AWS IoT サービスとデバイス（AWS Cloud9）のセットアップ</a></span></li><li><span><a href="#ノートブックインスタンスの-IAM-ロールに権限を追加" data-toc-modified-id="ノートブックインスタンスの-IAM-ロールに権限を追加-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>ノートブックインスタンスの IAM ロールに権限を追加</a></span></li><li><span><a href="#Lambda-関数が使用するコンテナイメージを作成" data-toc-modified-id="Lambda-関数が使用するコンテナイメージを作成-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Lambda 関数が使用するコンテナイメージを作成</a></span><ul class="toc-item"><li><span><a href="#コンテナイメージ更新用-Lambda-関数" data-toc-modified-id="コンテナイメージ更新用-Lambda-関数-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>コンテナイメージ更新用 Lambda 関数</a></span></li><li><span><a href="#モデルコンパイル用-Lambda-関数" data-toc-modified-id="モデルコンパイル用-Lambda-関数-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>モデルコンパイル用 Lambda 関数</a></span></li><li><span><a href="#デプロイ用-Lambda-関数" data-toc-modified-id="デプロイ用-Lambda-関数-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>デプロイ用 Lambda 関数</a></span></li></ul></li><li><span><a href="#Lambda-関数の作成と権限の設定" data-toc-modified-id="Lambda-関数の作成と権限の設定-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Lambda 関数の作成と権限の設定</a></span><ul class="toc-item"><li><span><a href="#タイムアウト時間の設定" data-toc-modified-id="タイムアウト時間の設定-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>タイムアウト時間の設定</a></span></li><li><span><a href="#アクセス権限の設定" data-toc-modified-id="アクセス権限の設定-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>アクセス権限の設定</a></span></li></ul></li><li><span><a href="#Step-Functions-Data-Science-SDK-でワークフローを作成" data-toc-modified-id="Step-Functions-Data-Science-SDK-でワークフローを作成-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Step Functions Data Science SDK でワークフローを作成</a></span><ul class="toc-item"><li><span><a href="#Step-Functions-の実行ロールの作成" data-toc-modified-id="Step-Functions-の実行ロールの作成-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Step Functions の実行ロールの作成</a></span></li><li><span><a href="#AWS-Step-Functions-ワークフローの作成" data-toc-modified-id="AWS-Step-Functions-ワークフローの作成-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>AWS Step Functions ワークフローの作成</a></span></li><li><span><a href="#Choice-State-と-Wait-State-の作成" data-toc-modified-id="Choice-State-と-Wait-State-の作成-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Choice State と Wait State の作成</a></span></li><li><span><a href="#Fail-状態の作成" data-toc-modified-id="Fail-状態の作成-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Fail 状態の作成</a></span></li><li><span><a href="#Workflow-の作成" data-toc-modified-id="Workflow-の作成-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Workflow の作成</a></span></li></ul></li><li><span><a href="#エッジ推論で使用する学習済みモデルの準備" data-toc-modified-id="エッジ推論で使用する学習済みモデルの準備-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>エッジ推論で使用する学習済みモデルの準備</a></span></li><li><span><a href="#デプロイ設定ファイルのセットアップ" data-toc-modified-id="デプロイ設定ファイルのセットアップ-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>デプロイ設定ファイルのセットアップ</a></span><ul class="toc-item"><li><span><a href="#ユーティリティ関数の定義" data-toc-modified-id="ユーティリティ関数の定義-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>ユーティリティ関数の定義</a></span></li><li><span><a href="#デプロイ関連情報の定義" data-toc-modified-id="デプロイ関連情報の定義-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>デプロイ関連情報の定義</a></span></li><li><span><a href="#学習済みモデルとアーティファクトファイルを-S3-にアップロード" data-toc-modified-id="学習済みモデルとアーティファクトファイルを-S3-にアップロード-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>学習済みモデルとアーティファクトファイルを S3 にアップロード</a></span></li><li><span><a href="#設定ファイル（yaml-形式）をファイルに保存" data-toc-modified-id="設定ファイル（yaml-形式）をファイルに保存-7.4"><span class="toc-item-num">7.4&nbsp;&nbsp;</span>設定ファイル（yaml 形式）をファイルに保存</a></span></li><li><span><a href="#設定ファイルを-S3-にアップロード" data-toc-modified-id="設定ファイルを-S3-にアップロード-7.5"><span class="toc-item-num">7.5&nbsp;&nbsp;</span>設定ファイルを S3 にアップロード</a></span></li></ul></li><li><span><a href="#AWS-Step-Functions-ワークフローの実行" data-toc-modified-id="AWS-Step-Functions-ワークフローの実行-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>AWS Step Functions ワークフローの実行</a></span><ul class="toc-item"><li><span><a href="#ワークフローの実行" data-toc-modified-id="ワークフローの実行-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>ワークフローの実行</a></span></li><li><span><a href="#既存のワークフローを呼び出して実行" data-toc-modified-id="既存のワークフローを呼び出して実行-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>既存のワークフローを呼び出して実行</a></span></li></ul></li><li><span><a href="#デプロイされたモデルの情報を一覧表示" data-toc-modified-id="デプロイされたモデルの情報を一覧表示-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>デプロイされたモデルの情報を一覧表示</a></span></li><li><span><a href="#[重要]-リソースの削除" data-toc-modified-id="[重要]-リソースの削除-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>[重要] リソースの削除</a></span></li></ul></div>

## [事前準備] AWS IoT サービスとデバイス（AWS Cloud9）のセットアップ

このノートブックでは、デバイスとして AWS Cloud9 を使用します。[こちらのワークショップコンテンツ](https://greengrassv2.workshop.aws/ja/) の、以下の部分を、このノートブックの実行を始める前に実施してください。

1. 「1. GREENGRASSを動かす環境の用意」すべてを実施
  - 「ディスク容量の拡張」の部分でディスク容量拡張コマンドを実行した際にエラーが出た場合、1、2分待ってから再度実行するとうまくいくことがあります
1. 「2. GREENGRASSのセットアップ」すべてを実施
  - （同一アカウントの複数名で本ノートブックを実行する場合）「2.2 GREENGRASSのセットアップ」の初めにある「Greengrassコアデバイスのセットアップ」の手順を以下のとおり変更してください
    - 「ステップ 1: Greengrass コアデバイスを登録する」のコアデバイスにご自身の名前など他のデバイスと区別可能な文字列を入れる
    - 「ステップ 2: モノのグループに追加して継続的なデプロイを適用する」の「モノのグループ」で「グループなし」を選択する
  - 「2.2 GREENGRASSのセットアップ」で環境変数を設定する手順では、ご自身の IAM ユーザの認証情報をご利用ください
    - AWS マネジメントコンソールの左上にあるサービスの検索欄にIAMと入力し、IAM サービスを選択します
    - IAM ダッシュボード画面にて、左のペインにあるユーザーをクリックします
    - お客様が用いている IAM ユーザー名をクリックします (IAM アカウントが無い場合はこちらを参考に作成してください)
    - ユーザー管理画面から認証情報タブを開きます
    - アクセスキーの作成をクリックします
    - アクセスキー ID とシークレットアクセスキーをコピーし、ローカルに保存します。こちらがお客様の IAM アカウントの認証情報です。
1. 「5.1 (CASE1) コンポーネントの作成とデプロイの準備」の「S3バケットの作成」までを実施
  - 同一アカウントの複数名で本ノートブックを実行する場合、バケットをひとつのみ作成しそれを共有する形でも構いません
1. （追加手順）IAM のコンソールの左側のメニューから「アクセス管理」->「ロール」をクリックし、検索窓に「GreengrassV2TokenExchangeRole」と入力して「GreengrassV2TokenExchangeRole」を開く
1. （追加手順）「ポリシーをアタッチします」をクリックして `AmazonS3FullAccess` をアタッチ
---


## ノートブックインスタンスの IAM ロールに権限を追加

以下の手順を実行して、ノートブックインスタンスに紐づけられた IAM ロールに、AWS Step Functions のワークフローを作成して実行するための権限と Amazon ECR にイメージを push するための権限を追加してください。

1. [Amazon SageMaker console](https://console.aws.amazon.com/sagemaker/) を開く
1. **ノートブックインスタンス** を開いて現在使用しているノートブックインスタンスを選択する
1. **アクセス許可と暗号化** の部分に表示されている IAM ロールへのリンクをクリックする
1. IAM ロールの ARN は後で使用するのでメモ帳などにコピーしておく
1. **ポリシーをアタッチします** をクリックして `AWSStepFunctionsFullAccess` を検索する
1. `AWSStepFunctionsFullAccess` の横のチェックボックスをオンにする
1. 同様の手順で以下のポリシーのチェックボックスをオンにして **ポリシーのアタッチ** をクリックする
    - `AmazonEC2ContainerRegistryFullAccess`
    - `AWSGreengrassFullAccess`
    - `AWSIoTFullAccess`
    - `IAMFullAccess`
    - `AWSLambda_FullAccess`

もしこのノートブックを SageMaker のノートブックインスタンス以外で実行している場合、その環境で AWS CLI 設定を行ってください。詳細は [Configuring the AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html) をご参照ください。

---
以下のセルの下から二番目の行の `<BUCKET_NAME>` に事前準備で作成した S3 バケット名（ggv2-workshop-xxx）を記載してから実行してください。同一アカウントで複数人でこのノートブックを実行する場合は一番下の行の `user_name` を各自のお名前で置き換えてください。

In [ ]:
import boto3
import yaml
import pandas as pd
from dateutil import tz
import sagemaker
import os
JST = tz.gettz('Asia/Tokyo')

region = boto3.session.Session().region_name
s3_client = boto3.client('s3', region_name=region)
account_id = boto3.client('sts').get_caller_identity().get('Account')
ggv2_client = boto3.client('greengrassv2', region_name=region)
iot_client = boto3.client('iot', region_name=region)
lambda_client = boto3.client('lambda', region_name=region)
ecr_client = boto3.client('ecr', region_name=region)
s3 = boto3.resource('s3')
sagemaker_session = sagemaker.Session()
sagemaker_role = sagemaker.get_execution_role()
bucket_name = '<BUCKET_NAME>'
user_name = 'sample'

## Lambda 関数が使用するコンテナイメージを作成

ここからは、3つのコンテナイメージを作成していきます。

それぞれの Lambda 関数では、デフォルトの Lambda 環境にないライブラリなどを使用するため、コンテナイメージごとソースコードを Lambda 関数にデプロイします。

### コンテナイメージ更新用 Lambda 関数

まずは、コンテナイメージ更新用 Lambda 関数が使用するコンテナイメージを作成します。この関数は、デプロイ用の Lambda 関数で使用するコンテナイメージを、ワークフロー実行時に設定したパラメタに応じて切り替えるためので、デプロイ用 Lambda 関数が使用するコンテナイメージを変えることのないユースケースであれば不要です。

Dockerfile と、Lambda 内で動かすスクリプトファイルを作成します。

In [ ]:
!mkdir -p docker/lambda-update/app

In [ ]:
%%writefile ./docker/lambda-update/Dockerfile

FROM public.ecr.aws/lambda/python:3.8

RUN pip3 install --upgrade pip
RUN pip3 install -qU boto3 pyyaml

COPY app/app.py   ./
CMD ["app.handler"]      

In [ ]:
%%writefile ./docker/lambda-update/app/app.py

import json
import boto3
import yaml

def handler(event, context):
    config_path = event['configPath']
    bucket_name = config_path.split('/')[2]
    object_key = config_path[len(bucket_name)+6:]
    s3 = boto3.resource('s3')

    bucket = s3.Bucket(bucket_name)
    obj = bucket.Object(object_key)

    response = obj.get()    
    body = response['Body'].read()
    
    config = yaml.safe_load(body)
    
    container_image = config['lambda-container']
    function_name = config['lambda-deploy-function']
    
    lambda_client = boto3.client('lambda')
    
    response = lambda_client.update_function_code(
        FunctionName=function_name,
        ImageUri=container_image
    )
    return {
        'statusCode': 200,
        'configPath': config_path,
        'response': response
    }

必要なファイルを作成できたので、コンテナイメージをビルドして Amazon ECR にプッシュします。

In [ ]:
import datetime

ecr_repository_lambda_update = 'lambda-update-continer-' + user_name
uri_suffix = 'amazonaws.com'

tag = ':' + datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9))).strftime('%Y%m%d-%H%M%S')
lambda_update_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository_lambda_update + tag)
print(lambda_update_repository_uri)

# Create ECR repository and push docker image
!docker build -t {ecr_repository_lambda_update + tag} docker/lambda-update
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com
!aws ecr create-repository --repository-name $ecr_repository_lambda_update
!docker tag {ecr_repository_lambda_update + tag} $lambda_update_repository_uri
!docker push $lambda_update_repository_uri

### モデルコンパイル用 Lambda 関数

次に、学習済みモデルを Amazon SageMaker Neo でコンパイルする Lambda 関数用のコンテナイメージを作成します。流れは先ほどと同様です。

In [ ]:
!mkdir -p docker/lambda-compile/app

In [ ]:
%%writefile ./docker/lambda-compile/Dockerfile

FROM public.ecr.aws/lambda/python:3.8

RUN pip3 install --upgrade pip
RUN pip3 install -qU boto3 pyyaml

COPY app/app.py   ./
CMD ["app.handler"]      

In [ ]:
%%writefile ./docker/lambda-compile/app/app.py

import json
import boto3
import yaml
import datetime
import time
import os

def handler(event, context):
    print(event)
    config_path = event['configPath']
    
    bucket_name = config_path.split('/')[2]
    object_key = config_path[len(bucket_name)+6:]
    s3 = boto3.resource('s3')
    region = boto3.session.Session().region_name
    sagemaker_client = boto3.client('sagemaker', region_name=region)
    lambda_client = boto3.client('lambda')

    bucket = s3.Bucket(bucket_name)
    obj = bucket.Object(object_key)

    response = obj.get()    
    body = response['Body'].read()
    
    config = yaml.safe_load(body)
    
    if config['model-information']['compile-model']:
    
        framework = config['model-information']['framework']

        timestamp = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9))).strftime('%Y%m%d-%H%M%S')
        job_name = framework + '-compile-' + timestamp
        model_s3_path = config['model-information']['original-model-path']
        input_name = config['model-information']['input-name']
        input_shape = config['model-information']['input-shape']
        output_location = config['model-information']['compiled-model-path']
        target_device = config['model-information']['target-device']

        response = lambda_client.get_function(
            FunctionName=os.environ['AWS_LAMBDA_FUNCTION_NAME']
        )
        role = response['Configuration']['Role']
        data_input_config = '{"' + input_name + '":'+input_shape + '}'

        response = sagemaker_client.create_compilation_job(
            CompilationJobName=job_name,
            RoleArn=role,
            InputConfig={
                'S3Uri': model_s3_path,
                'DataInputConfig':data_input_config,
                'Framework': framework
            },
            OutputConfig={
                'S3OutputLocation': output_location,
                                                'TargetDevice':target_device
            },
            StoppingCondition={ 'MaxRuntimeInSeconds': 9000 }    
        )
        
        time.sleep(60)
        return {
            'statusCode': 200,
            'configPath': config_path,
            'compileJobName': job_name,
            'response': response
        }
    else:
        return {
            'statusCode': 200,
            'configPath': config_path
        }

In [ ]:
import datetime

ecr_repository_lambda_compile = 'lambda-compile-model-' + user_name
uri_suffix = 'amazonaws.com'

tag = ':' + datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9))).strftime('%Y%m%d-%H%M%S')
lambda_compile_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository_lambda_compile + tag)
print(lambda_compile_repository_uri)

# Create ECR repository and push docker image
!docker build -t {ecr_repository_lambda_compile + tag} docker/lambda-compile
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository_lambda_compile
!docker tag {ecr_repository_lambda_compile + tag} $lambda_compile_repository_uri
!docker push $lambda_compile_repository_uri

### デプロイ用 Lambda 関数

最後に、デプロイ用 Lambda 関数が使用するコンテナイメージを作成します。

In [ ]:
!mkdir -p docker/lambda/app

In [ ]:
%%writefile ./docker/lambda/Dockerfile

FROM public.ecr.aws/lambda/python:3.8

RUN pip3 install --upgrade pip
RUN pip3 install -qU boto3 pyyaml

COPY app/app.py   ./
CMD ["app.handler"]      

In [ ]:
%%writefile ./docker/lambda/app/app.py

import json
import boto3
import yaml
import os
import time

region = boto3.session.Session().region_name
sagemaker_client = boto3.client('sagemaker', region_name=region)
ggv2_client = boto3.client('greengrassv2', region_name=region)
    
def generate_recipe(artifact_files, component_name, component_version, 
                    model_name, entrypoint_script_name, pip_libraries, model_input_shape):
    # create artifact list
    artifacts = ''
    for f in artifact_files:
        artifacts += f'          - URI: {f}\n'

    # define recipe
    artifacts_path = '{artifacts:path}'
    recipe = f"""---
RecipeFormatVersion: '2020-01-25'
ComponentName: {component_name}
ComponentVersion: {component_version}
ComponentType: "aws.greengrass.generic"
ComponentDescription: Publish MQTT message to AWS IoT Core in Docker image.
ComponentPublisher: Amazon
ComponentConfiguration:
  DefaultConfiguration:
    accessControl:
      aws.greengrass.ipc.pubsub:
        com.example.Publisher:pubsub:1:
          policyDescription: "Allows access to publish to all topics."
          operations:
          - "aws.greengrass#PublishToTopic"
          resources:
          - "*"
Manifests:
  - Platform:
      os: "linux"
      architecture: "amd64"
    Name: "Linux"
    Lifecycle:
      Install:
        RequiresPrivilege: true
        Script: "pip3 install --upgrade pip\\n\
          pip3 install {pip_libraries}\\n\
          pip3 install awsiotsdk numpy\\n\
          apt-get install -y libgl1-mesa-dev\\n\
          mkdir {artifacts_path}/model\\n\
          tar xf {artifacts_path}/{model_name} -C {artifacts_path}/model"
      Run: "python3 {artifacts_path}/{entrypoint_script_name} {artifacts_path} '{model_input_shape}'"
    Artifacts:
{artifacts}
    """
    return recipe

def deploy_component(target_arn, deployment_name, components):
    response = ggv2_client.create_deployment(
        targetArn=target_arn, # デプロイ先のIoT thing か group
        deploymentName=deployment_name, # デプロイの名前
        components=components,
        iotJobConfiguration={
            'timeoutConfig': {'inProgressTimeoutInMinutes': 600}
        },
        deploymentPolicies={
            'failureHandlingPolicy': 'ROLLBACK',
            'componentUpdatePolicy': {
                'timeoutInSeconds': 600,
                'action': 'NOTIFY_COMPONENTS'
            },
            'configurationValidationPolicy': {
                'timeoutInSeconds': 600
            }
        },
        tags={
            'Name': deployment_name
        }
    )
    return response

def get_deployment_info(deployment_name):
    deployments = ggv2_client.list_deployments()['deployments']

    deployment_id = ''
    group_id = ''
    for d in deployments:
        if d['deploymentName'] == deployment_name:
            deployment_id = d['deploymentId']
            group_arn = d['targetArn']
            return deployment_id, group_arn
        
    return -1, -1

def run_deployment(group_arn, deployment_name, component_name, component_version):
    components={
        component_name: { # コンポーネントの名前
            'componentVersion': component_version,
            'runWith': {
                'posixUser': 'root'
            }
        },
    #     'aws.greengrass.Cli': { # コンポーネントの名前
    #         'componentVersion': '2.3.0',
    #     },

    }

    response = deploy_component(group_arn, deployment_name, components)
    
    return response

def handler(event, context):
    print(event)
    config_path = event['Payload']['Payload']['configPath']
    bucket_name = config_path.split('/')[2]
    object_key = config_path[len(bucket_name)+6:]
    s3 = boto3.resource('s3')

    bucket = s3.Bucket(bucket_name)
    obj = bucket.Object(object_key)

    response = obj.get()    
    body = response['Body'].read()
    
    config = yaml.safe_load(body)
    
    component_name = config['component-name']
    component_version = config['component-version']
    deployment_name = config['deployment-name']
    entrypoint_script_name = config['entrypoint-script-name']
    pip_libraries = config['pip-libraries']
    model_path = config['model-information']['original-model-path']
    compile_model = config['model-information']['compile-model']
    artifact_files = config['artifact-files']
    
    if compile_model:
        compile_job_name = event['Payload']['Payload']['compileJobName']
        print(compile_job_name)

        while True:
            status = sagemaker_client.describe_compilation_job(CompilationJobName=compile_job_name)['CompilationJobStatus']
            if status == 'COMPLETED' or status == 'FAILED':
                print(status)
                break
            time.sleep(30)
    
        model_path = sagemaker_client.describe_compilation_job(CompilationJobName=compile_job_name)['ModelArtifacts']['S3ModelArtifacts']
        
    model_name = os.path.basename(model_path)
    artifact_files.append(model_path)
    recipe = generate_recipe(artifact_files, component_name, component_version,
                             model_name, entrypoint_script_name, pip_libraries, config['model-information']['input-shape'])
    print(recipe)
    print(type(recipe.encode()))
    
    # コンポーネント作成
    response = ggv2_client.create_component_version(
        inlineRecipe=recipe.encode('utf-8')
    )
    component_vesrion_arn = response['arn']
    
    deployment_id, group_arn = get_deployment_info(deployment_name)

    response = run_deployment(group_arn, deployment_name, component_name, component_version)

    return {
        'statusCode'        : 200,
        'component-name': component_name,
        'response': response
    }

In [ ]:
import datetime

ecr_repository_lambda_deploy = 'lambda-deploy-gg-' + user_name
uri_suffix = 'amazonaws.com'

deploy_lambda_container_tag = ':' + datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9))).strftime('%Y%m%d-%H%M%S')
lambda_deploy_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository_lambda_deploy + deploy_lambda_container_tag)
print(lambda_deploy_repository_uri)

# Create ECR repository and push docker image
!docker build -t {ecr_repository_lambda_deploy + deploy_lambda_container_tag} docker/lambda
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com
!aws ecr create-repository --repository-name $ecr_repository_lambda_deploy
!docker tag {ecr_repository_lambda_deploy + deploy_lambda_container_tag} $lambda_deploy_repository_uri
!docker push $lambda_deploy_repository_uri

## Lambda 関数の作成と権限の設定

コンテナ更新用 (update-container)、モデルコンパイル用 (compile-model)、コンポーネントのデプロイ用 (deploy-components-to-device) の Lambda 関数をそれぞれ作成します。下にある2つのセルでは、関数の作成と以下の操作を API を使って行なっています。

### タイムアウト時間の設定

全ての処理が終わるまで関数がタイムアウトしないように、それぞれの関数のタイムアウト時間をすべて 5分に設定します。

### アクセス権限の設定

それぞれの関数の権限を以下のように設定します。<br>**今回は AmazonS3FullAccess など強い権限を付与していますが、実際の環境で使用する場合は必要最小限の権限のみを追加するようにしてください。**

**update-container 関数**
- ロールに以下のポリシーを追加
  -  AmazonS3FullAccess
  - AWSLambda_FullAccess
  
**compile-model 関数**
- ロールに以下のポリシーを追加
  -  AmazonS3FullAccess
  - AWSLambda_FullAccess
  - AmazonSageMakerFullAccess
- 「信頼関係」に以下を設定
```
"Service": [
          "sagemaker.amazonaws.com",
          "lambda.amazonaws.com"
        ]
```


**deploy-components-to-device 関数**
- ロールに以下のポリシーを追加
  -  AmazonS3FullAccess
  - AmazonSageMakerFullAccess
  - AWSIoTFullAccess
  - AWSGreengrassFullAccess
- 「信頼関係」に以下を設定
```
"Service": [
          "sagemaker.amazonaws.com",
          "lambda.amazonaws.com"
        ]
```

---

In [ ]:
import boto3
import json
from datetime import datetime
from dateutil import tz
from time import sleep
JST = tz.gettz('Asia/Tokyo')


iam_client = boto3.client('iam')

def create_container_lambda_function(function_name, image_uri, policy_list, trust_service_list=[]):

    timestamp = datetime.now(tz=JST).strftime('%Y%m%d-%H%M%S')
    lambda_function_name = function_name
    lambda_inference_policy_name = lambda_function_name + '-policy-'+timestamp
    lambda_inference_role_name = lambda_function_name + '-role-'+timestamp

    inline_policy = {
        'Version': '2012-10-17',
        'Statement': [
            {
                'Effect': 'Allow',
                'Action': 'logs:CreateLogGroup',
                'Resource': f'arn:aws:logs:{region}:{account_id}:*'
            },
            {
                'Effect': 'Allow',
                'Action': [
                    'logs:CreateLogStream',
                    'logs:PutLogEvents'
                ],
                'Resource': [
                    f'arn:aws:logs:{region}:{account_id}:log-group:/aws/lambda/{lambda_function_name}:*'
                ]
            }
        ]
    }

    response = iam_client.create_policy(
        PolicyName=lambda_inference_policy_name,
        PolicyDocument=json.dumps(inline_policy),
    )

    policy_arn = response['Policy']['Arn']

    service_list = ["lambda.amazonaws.com"]
    for t in trust_service_list:
        service_list.append(t)
    
    assume_role_policy = {
      "Version": "2012-10-17",
      "Statement": [{"Sid": "","Effect": "Allow","Principal": {"Service":service_list},"Action": "sts:AssumeRole"}]
    }
    response = iam_client.create_role(
        Path = '/service-role/',
        RoleName = lambda_inference_role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy),
        MaxSessionDuration=3600*12 # 12 hours
    )
    
    lambda_role_arn = response['Role']['Arn']
    lambda_role_name = response['Role']['RoleName']
    
    response = iam_client.attach_role_policy(
        RoleName=lambda_inference_role_name,
        PolicyArn=policy_arn
    )
    
    for p in policy_list:
        arn = 'arn:aws:iam::aws:policy/' + p
        response = iam_client.attach_role_policy(
            RoleName=lambda_inference_role_name,
            PolicyArn=arn
        )
    
    sleep(20) # wait until IAM is created
    
    response = lambda_client.create_function(
        FunctionName=function_name,
        Role=lambda_role_arn,
        Code={
            'ImageUri':image_uri
        },
        Timeout=60*5, # 5 minutes
        MemorySize=128, # 128 MB
        Publish=True,
        PackageType='Image',
    )
    
    return lambda_role_name, policy_arn

In [ ]:
lambda_policies = []
lambda_roles = []

lambda_function_name_update = 'update-container-' + user_name
policy_list = ['AmazonS3FullAccess', 'AWSLambda_FullAccess']
r, p = create_container_lambda_function(lambda_function_name_update, lambda_update_repository_uri, policy_list)
lambda_policies.append(p)
lambda_roles.append(r)

lambda_function_name_compile = 'compile-model-' + user_name
policy_list = ['AmazonS3FullAccess', 'AWSLambda_FullAccess', 'AmazonSageMakerFullAccess']
trust_list = ["sagemaker.amazonaws.com"]
r, p = create_container_lambda_function(lambda_function_name_compile, lambda_compile_repository_uri, policy_list, trust_list)
lambda_policies.append(p)
lambda_roles.append(r)

lambda_function_name_deploy = 'deploy-components-to-device-' + user_name
policy_list = ['AmazonS3FullAccess', 'AWSIoTFullAccess', 'AmazonSageMakerFullAccess', 'AWSGreengrassFullAccess']
r, p = create_container_lambda_function(lambda_function_name_deploy, lambda_deploy_repository_uri, policy_list, trust_list)
lambda_policies.append(p)
lambda_roles.append(r)

## Step Functions Data Science SDK でワークフローを作成

Step Functions で使用する実行ロールを作成します。

### Step Functions の実行ロールの作成

 作成した Step Functions ワークフローは、AWS の他のサービスと連携するための IAM ロールを必要とします。以下のセルを実行して、必要な権限を持つ IAM Policy を作成し、それを新たに作成した IAM Role にアタッチします。
 
なお、今回は広めの権限を持つ IAM Policy を作成しますが、ベストプラクティスとしては必要なリソースのアクセス権限と必要なアクションのみを有効にします。

In [ ]:
step_functions_policy_name = 'AmazonSageMaker-StepFunctionsWorkflowExecutionPolicy-' + user_name
inline_policy ={
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": [
                "events:PutTargets",
                "events:DescribeRule",
                "events:PutRule"
            ],
            "Resource": [
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTrainingJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTransformJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTuningJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForECSTaskRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForBatchJobsRule"
            ]
        },
        {
            "Sid": "VisualEditor1",
            "Effect": "Allow",
            "Action": "iam:PassRole",
            "Resource": sagemaker_role,
            "Condition": {
                "StringEquals": {
                    "iam:PassedToService": "sagemaker.amazonaws.com"
                }
            }
        },
        {
            "Sid": "VisualEditor2",
            "Effect": "Allow",
            "Action": [
                "batch:DescribeJobs",
                "batch:SubmitJob",
                "batch:TerminateJob",
                "dynamodb:DeleteItem",
                "dynamodb:GetItem",
                "dynamodb:PutItem",
                "dynamodb:UpdateItem",
                "ecs:DescribeTasks",
                "ecs:RunTask",
                "ecs:StopTask",
                "glue:BatchStopJobRun",
                "glue:GetJobRun",
                "glue:GetJobRuns",
                "glue:StartJobRun",
                "lambda:InvokeFunction",
                "sagemaker:CreateEndpoint",
                "sagemaker:CreateEndpointConfig",
                "sagemaker:CreateHyperParameterTuningJob",
                "sagemaker:CreateModel",
                "sagemaker:CreateProcessingJob",
                "sagemaker:CreateTrainingJob",
                "sagemaker:CreateTransformJob",
                "sagemaker:DeleteEndpoint",
                "sagemaker:DeleteEndpointConfig",
                "sagemaker:DescribeHyperParameterTuningJob",
                "sagemaker:DescribeProcessingJob",
                "sagemaker:DescribeTrainingJob",
                "sagemaker:DescribeTransformJob",
                "sagemaker:ListProcessingJobs",
                "sagemaker:ListTags",
                "sagemaker:StopHyperParameterTuningJob",
                "sagemaker:StopProcessingJob",
                "sagemaker:StopTrainingJob",
                "sagemaker:StopTransformJob",
                "sagemaker:UpdateEndpoint",
                "sns:Publish",
                "sqs:SendMessage"
            ],
            "Resource": "*"
        }
    ]
}

response = iam_client.create_policy(
    PolicyName=step_functions_policy_name,
    PolicyDocument=json.dumps(inline_policy),
)

step_functions_policy_arn = response['Policy']['Arn']

In [ ]:
step_functions_role_name = 'AmazonSageMaker-StepFunctionsWorkflowExecutionRole-' + user_name
assume_role_policy = {
      "Version": "2012-10-17",
      "Statement": [{"Sid": "","Effect": "Allow","Principal": {"Service":"states.amazonaws.com"},"Action": "sts:AssumeRole"}]
    }
response = iam_client.create_role(
    Path = '/service-role/',
    RoleName = step_functions_role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy),
    MaxSessionDuration=3600*12 # 12 hours
)

step_functions_role_arn = response['Role']['Arn']

response = iam_client.attach_role_policy(
    RoleName=step_functions_role_name,
    PolicyArn=step_functions_policy_arn
)

response = iam_client.attach_role_policy(
    RoleName=step_functions_role_name,
    PolicyArn='arn:aws:iam::aws:policy/CloudWatchEventsFullAccess'
)


In [ ]:
workflow_execution_role = step_functions_role_arn

In [ ]:
import sys
!{sys.executable} -m pip install -qU "stepfunctions==2.1.0"

In [ ]:
import stepfunctions
from stepfunctions import steps
from stepfunctions.inputs import ExecutionInput
from stepfunctions.steps import (
    Chain,
    ChoiceRule,
    ModelStep,
    ProcessingStep,
    TrainingStep,
    TransformStep,
)
# from stepfunctions.template import TrainingPipeline
from stepfunctions.template.utils import replace_parameters_with_jsonpath
from stepfunctions.workflow import Workflow

### AWS Step Functions ワークフローの作成

まずは、AWS Step Functions ワークフロー実行時に指定するパラメタの定義をします。

In [ ]:
execution_input = ExecutionInput(
    schema={
        "ConfigFilePath": str,
    }
)

ここからは、先ほど作成した 3つの Lambda 関数を順に実行するようなワークフローを作成していきます。

In [ ]:
from stepfunctions.steps.states import Retry
lambda_update_step = stepfunctions.steps.compute.LambdaStep(
    "Update Container Image",
    parameters={
        "FunctionName": lambda_function_name_update,
        "Payload": {
            "configPath": execution_input["ConfigFilePath"],
        },
    },
)

lambda_compile_step = stepfunctions.steps.compute.LambdaStep(
    "Compile Model",
    parameters={
        "FunctionName": lambda_function_name_compile,
        "Payload": {
            "configPath": execution_input["ConfigFilePath"],
        },
    },
)

lambda_deploy_step = stepfunctions.steps.compute.LambdaStep(
    "Deploy components",
    parameters={
        "FunctionName": lambda_function_name_deploy,
        "Payload": {
            "Payload.$": "$"
        },
    },
)
lambda_update_step.add_retry(
    Retry(error_equals=["States.TaskFailed"], interval_seconds=15, max_attempts=2, backoff_rate=4.0)
)
lambda_compile_step.add_retry(
    Retry(error_equals=["States.TaskFailed"], interval_seconds=15, max_attempts=2, backoff_rate=4.0)
)
lambda_deploy_step.add_retry(
    Retry(error_equals=["States.TaskFailed"], interval_seconds=15, max_attempts=2, backoff_rate=4.0)
)

### Choice State と Wait State の作成

設定ファイルの `neo-compile` が True か False かによって、モデルコンパイル用 Lambda 関数実行後の待ち時間を調整するために、Choice State と Wait State を使用します。

In [ ]:
wait_5_step =  stepfunctions.steps.states.Wait(
    "Wait for five minutes",
    seconds = 300,
)

wait_2_step =  stepfunctions.steps.states.Wait(
    "Wait for two minutes",
    seconds = 120,
)

wait_choice_step = stepfunctions.steps.states.Choice(
    "Compile enable"
)
wait_choice_step.add_choice(
    rule=ChoiceRule.IsPresent(variable=lambda_compile_step.output()["Payload"]["response"], value=True),
    next_step=wait_5_step
)
wait_choice_step.default_choice(
    next_step=wait_2_step
)
wait_2_step.next(lambda_deploy_step)
wait_5_step.next(lambda_deploy_step)

### Fail 状態の作成
いずれかのステップが失敗したときにワークフローが失敗だとわかるように Fail 状態を作成します。

エラーハンドリングのために [Catch Block](https://aws-step-functions-data-science-sdk.readthedocs.io/en/stable/states.html#stepfunctions.steps.states.Catch) を使用します。もし いずれかの Step が失敗したら、Fail 状態に遷移します。

In [ ]:
failed_state_sagemaker_processing_failure = stepfunctions.steps.states.Fail(
    "ML Workflow failed", cause="Failed"
)
catch_state_processing = stepfunctions.steps.states.Catch(
    error_equals=["States.TaskFailed"],
    next_step=failed_state_sagemaker_processing_failure,
)

lambda_update_step.add_catch(catch_state_processing)
lambda_compile_step.add_catch(catch_state_processing)
lambda_deploy_step.add_catch(catch_state_processing)

### Workflow の作成

ここまでで Step Functions のワークフローを作成する準備が完了しました。`branching_workflow.create()` を実行することで、ワークフローが作成されます。一度作成したワークフローを更新する場合は `branching_workflow.update()` を実行します。

Chain を使って各 Step を連結してワークフローを作成します。既存のワークフローを変更する場合は、update() を実行します。ログに ERROR が表示された場合は、以下のセルを再度実行してください。

In [ ]:
import time
workflow_graph = Chain([lambda_update_step, lambda_compile_step, wait_choice_step])

branching_workflow = Workflow(
    name="gg-deploy-workflow-" + user_name,
    definition=workflow_graph,
    role=workflow_execution_role,
)

branching_workflow.create()
branching_workflow.update(workflow_graph)
time.sleep(5)
branching_workflow.render_graph(portrait=False)

ここまででモデルデプロイワークフローが作成できました。ワークフローは初めにいったん作成してしまえば、あとは実行するだけです。

ここからの手順は、新しいモデルをエッジデバイスにデプロイするたびに実行する想定です。

---

## エッジ推論で使用する学習済みモデルの準備

このノートブックでは、Keras の学習済みの MobileNet モデルを使用します。Amazon SageMaker Neo で動作確認済みのモデルは [こちらのドキュメント](https://docs.aws.amazon.com/sagemaker/latest/dg/neo-supported-edge-tested-models.html) で確認できます。また、サポートされているフレームワークのバージョンは [こちらのドキュメント](https://docs.aws.amazon.com/sagemaker/latest/dg/neo-supported-devices-edge-frameworks.html) で確認できます。

以下のセルを実行して、学習済みの MobileNet モデルをダウンロードし、`mobilenet.h5` として保存します。また、`model.summary()` を実行して、入力レイヤ名を確認します。Keras の MobileNet であればたいてい `input_1` となります。この名前を後ほど設定ファイルに記載するので覚えておいてください。

自分で学習したモデルをデプロイする場合は、この手順は不要です。

In [ ]:
import tensorflow as tf

model = tf.keras.applications.MobileNet()
model.save('mobilenet.h5')
model.summary()

---

## デプロイ設定ファイルのセットアップ

このノートブックの構成では、作成した AWS Step Functions ワークフローを実行する際に、デプロイに関する情報を記載した設定ファイル（yaml 形式）を入力パラメタとして指定します。モデルを学習した際の学習ジョブの情報など、他に記録しておきたい情報があれば、この yaml ファイルに記述を追加してください。ここからは、設定ファイルを作成していきます。

### ユーティリティ関数の定義

まずは設定ファイルの作成に必要な関数を準備します。

In [ ]:
import boto3
# region = boto3.session.Session().region_name
# ggv2_client = boto3.client('greengrassv2', region_name=region)

def get_latest_component_version(component_name):
    component_list = ggv2_client.list_components()['components']
    for c in component_list:
        if c['componentName'] == component_name:
            return c['latestVersion']['componentVersion']
        
def get_target_group_arn(deployment_name):
    deployments = ggv2_client.list_deployments()['deployments']

    deployment_id = ''
    group_id = ''
    for d in deployments:
        if d['deploymentName'] == deployment_name:
            return d['targetArn']

    return -1
        
def increment_version(current_version, target='revision'):
    # target: major, minor, revision
    
    if current_version == None:
        return '0.0.1'
    major, minor, revision = list(map(int, current_version.split('.')))
    
    if target == 'revision':
        revision += 1
    elif target == 'minor':
        minor += 1
        revision = 0
    elif target == 'major':
        major += 1
        minor = 0
        revision = 0
    else:
        print('[ERROR] invalid target value')
        return current_version
    
    return '{}.{}.{}'.format(major, minor, revision)

### デプロイ関連情報の定義

以下のセルに、デプロイしたいコンポーネントの情報を入力して実行します。**必ず以下の変数をご自身の環境に合わせて書き換えてください。**

- `deployment_name`: [事前準備]の手順で作成したデバイスに紐づいたデプロイメント名


前の手順で確認したモデルの入力名が `input_1` でなかった場合は、`model_input_name` の値も書き換えてください。必要に応じて以下の変数を書き換えてください。

- Amazon SageMaker Neo 用の設定
  - `model_input_name`: 学習済みモデルの入力名。Keras の場合、model.summary() で確認する
  - `model_input_shape`: 学習済みモデルの入力サイズ。Keras の場合、model.summary() で確認する
  - `model_framework`: 学習済みモデルのフレームワーク
  - `target_device`: コンパイルターゲットデバイスの種類
- AWS IoT Greengrass V2 用の設定
  - `gg_s3_path`: 設定ファイルやアーティファクトを保存する S3 パス（このノートブックを実行するのと同じリージョンのバケットにしてください）
　　　- `component_name`: コンポーネントの名前（任意の名前）
  - `target_group_arm`: デプロイ先のターゲットグループの ARN
  - `pip_libraries`: Greengrass レシピに記載する、エッジ推論に必要なライブラリ名（複数ある場合はスペース区切り）
- その他の設定
  - `deploy_lambda_container`: デプロイ用 Lambda 関数で使用するコンテナイメージの URI（コンテナイメージ作成手順の中で Amazon ECR にプッシュしたイメージ）

このノートブックでは、`gg_s3_path` で設定した S3 パス以下に必要なファイルをアップロードします。また、モデルを SageMaker Neo でコンパイルするので、Greengrass コンポーネントに `pip install` するライブラリを指定するための `pip_libraries` を用意しています。

In [ ]:
gg_s3_path = 's3://'+bucket_name+'/gg/' + user_name # 設定ファイル、アーティファクトファイルなどを保存する S3 パス
component_name = 'com.example.IoTPublisher.' + user_name
deployment_name = 'Deployment for GreengrassQuickStartGroup'
target_group_arm = get_target_group_arn(deployment_name)
target_device = 'ml_m5'
model_input_name = 'input_1'
model_input_shape = '[1, 3, 224, 224]'
model_framework =  'KERAS'
deploy_lambda_container = lambda_deploy_repository_uri
pip_libraries = 'dlr pillow opencv-python opencv-contrib-python'

### 学習済みモデルとアーティファクトファイルを S3 にアップロード

デバイスにデプロイするサンプルとして用意したファイルを S3 にアップロードします。アーティファクトファイルは、`bucket_name`/gg/artifacts/コンポーネント名/コンポーネントバージョン にアップロードされます。

このサンプルノートブックでは、アーティファクトファイルに変更がなくても必ずコンポーネントバージョンごとにファイルを S3 に保存する構成になっています。

In [ ]:

!tar zcvf mobilenet.tar.gz mobilenet.h5

latest_version = get_latest_component_version( component_name)
component_version = increment_version(latest_version)

path = os.path.join('gg', user_name, 'artifacts', component_name, component_version, 'data')
artifacts_s3_path = sagemaker_session.upload_data(path='artifacts', bucket=bucket_name, key_prefix=path)

path = ('gg', user_name, 'models')
model_s3_path = sagemaker_session.upload_data(path='mobilenet.tar.gz', bucket=bucket_name, key_prefix=path)

print('new component version:', component_version)

### 設定ファイル（yaml 形式）をファイルに保存

以下のセルを実行して、設定ファイルを作成します。Greengrass コンポーネントのバージョンは、最新のバージョンのリビジョン番号を 1インクリメントしたものが自動的にセットされます。メジャー番号やマイナー番号をインクリメントしたい場合は、`get_latest_component_version` の引数に `target='major'` などを指定してください。

以下のセルでは、学習済みモデルが `{gg_s3_path}/models` という S3 パスに保存されている想定で設定ファイル（yaml 形式）を作成して保存しています。SageMaker 学習ジョブが出力したファイルを直接指定したい場合は、以下のセルの `original-model-path` にモデルが保存されているフルパスを設定してください。また、SageMaker Neo でコンパイル済みのモデルを指定する場合は、`compile-model:` に `False` を設定し、`compiled-model-path` にはコンパイル済みのモデルが保存されている S3 パスを設定してください。

In [ ]:
import datetime

timestamp = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9))).strftime('%Y%m%d-%H%M%S')

config_string = f"""
component-name: {component_name}
component-version: {component_version}
deployment-name: {deployment_name}
target-group-arn: {target_group_arm}
entrypoint-script-name: 'run.py'
pip-libraries: {pip_libraries}
model-information:
    original-model-path: {model_s3_path}
    compile-model: True
    compiled-model-path:  {gg_s3_path}/compiled-models/
    target-device: {target_device}
    input-name: {model_input_name}
    input-shape: '{model_input_shape}'
    framework: {model_framework}
lambda-container: {deploy_lambda_container}
lambda-deploy-function: 'deploy-components-to-device-{user_name}'
artifact-files: 
    - {artifacts_s3_path}/run.py
    - {artifacts_s3_path}/inference.py
    - {artifacts_s3_path}/classification-demo.png
    - {artifacts_s3_path}/image_net_labels.json
"""
config_name = timestamp + '-' + component_version + '.yaml'
with open(config_name, 'w') as f:
    f.write(config_string)

### 設定ファイルを S3 にアップロード

作成した設定ファイルを S3 にアップロードします。アップロート先の S3 パスが `upload_path` に保存されます。AWS Step Functions ワークフローを実行する際に、このファイルパスを入力パラメタとして指定します。

In [ ]:
import yaml
import os

with open(config_name) as file:
    config = yaml.safe_load(file)
    component_name = config['component-name']

upload_path = os.path.join(gg_s3_path, 'config', component_name, component_version, config_name)
    
!aws s3 cp $config_name $upload_path

## AWS Step Functions ワークフローの実行

それではいよいよ、ワークフローを実行して ML モデルやスクリプトをエッジにデプロイします。ワークフローの実行方法は 2通りあります。このノートブックを初めて使用した場合は、1の方法をご利用ください。

1. ワークフローを作成し、続けて実行する
1. 既存のワークフローを呼び出して実行する

### ワークフローの実行
以下のセルを実行してワークフローを開始します。引数の `ConfigFilePath` に設定ファイルが置いてある S3 パスが指定されています。このセルを実行した場合は、次のセルを実行する必要はありません。

過去に作成した設定ファイルを指定してワークフローを実行すれば、当時と同じ処理を実行することができます。

In [ ]:
execution = branching_workflow.execute(
    inputs={
        'ConfigFilePath': upload_path
    }
)

### 既存のワークフローを呼び出して実行

すでに作成してあるワークフローを実行する場合は以下のセルの `workflow_arn` にワークフローの ARN を入力し、コメントアウトを解除してから実行してください。ワークフローの ARN は [AWS Step Functions のコンソール](https://ap-northeast-1.console.aws.amazon.com/states/home?region=ap-northeast-1#/statemachines) から確認できます。

In [ ]:
# from stepfunctions.workflow import Workflow

# workflow_arn = 'arn:aws:states:ap-northeast-1:420964472730:stateMachine:gg-deploy-workflow'
# existing_workflow = Workflow.attach(workflow_arn)

# execution = existing_workflow.execute(
#     inputs={
#         'ConfigFilePath': upload_path
#     }
# )

以下のセルを実行すると、ワークフローの実行状況を確認できます。

In [ ]:
execution.render_progress()

デプロイが完了したら、Cloud9 のターミナルで以下のコマンドを実行してログを表示します。うまくデプロイできていれば、5秒ごとに推論結果が表示されます。なお、常に同じ画像を使って推論しているため、常に同じ結果が表示されます。

> tail -f /tmp/Greengrass_HelloWorld.log

## デプロイされたモデルの情報を一覧表示

どのデバイスにどのモデルがデプロイされたかを知りたいことがあります。その場合は、Greengrass の deployments のリストから各種情報を API やワークフロー実行時の設定ファイルを使って情報を一覧表示することができます。

まずは必要な関数を定義します。

In [ ]:
# import boto3
# import yaml
# import pandas as pd
# from dateutil import tz
# JST = tz.gettz('Asia/Tokyo')

# region = boto3.session.Session().region_name
# s3_client = boto3.client('s3', region_name=region)
# ggv2_client = boto3.client('greengrassv2', region_name=region)
# iot_client = boto3.client('iot', region_name=region)
# s3 = boto3.resource('s3')

def get_device_in_deployment_list():
    deployments = ggv2_client.list_deployments()['deployments']

    device_list = []
    for d in deployments:

        thing_group_name =  d['targetArn'].split('/')[-1]
        
        try:
            response = iot_client.list_things_in_thing_group(
                thingGroupName=thing_group_name
            )
            things = response['things']
        except:
            things = [thing_group_name]
        for thing in things:
            response = ggv2_client.get_core_device(
                coreDeviceThingName=thing
            )

            device_list.append({
                'deployment-name': d['deploymentName'],
                'target-arn': d['targetArn'],
                'thing-group-name': thing_group_name,
                'thing-name': thing,
                'status': response['status'],
                'last-status-updated': response['lastStatusUpdateTimestamp'].astimezone(JST)
            })
        
    return device_list

def get_installed_component_list(device_name):
    response = ggv2_client.list_installed_components(
    coreDeviceThingName=device_name
    )

    component_list = []
    for d in response['installedComponents']:
        component_list.append([d['componentName'], d['componentVersion']])
        
    return component_list

def get_config_file_name(bucket_name, prefix):

    prefix = prefix + '/'
    
    def search_component(latest, start_after):
        objects = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix, StartAfter=start_after)

        if "Contents" in objects:
            keys = [content["Key"][len(prefix):] for content in objects["Contents"]]
            for k in keys:
                if k > latest:
                    latest = k
            if objects.get("isTruncated"):
                return search_component(latest=latest, start_after=keys[-1])
        return latest
    
    return search_component('0', '')

def get_config_info_as_yaml(config_file_path, component_info):
    component_name, component_version = component_info
    bucket_name = config_file_path.split('/')[2]
    prefix = os.path.join(config_file_path[len(bucket_name)+6:] , 'config', component_name, component_version)
    config_file_name = os.path.join(prefix, get_config_file_name(bucket_name, prefix))

    bucket = s3.Bucket(bucket_name)
    obj = bucket.Object(config_file_name)
    try:
        response = obj.get()    
    except:
        return None
    
    body = response['Body'].read()
    config = yaml.safe_load(body)
    
    return config


def show_deployed_model_info(config_file_path):
    device_list = get_device_in_deployment_list()

    column_name = [
        'deployment-name',#0
        'target-group-arn', 
        'thing-name',
        'thing-status',
        'thing-last-status-updated',#5
        'component-name', 
        'deployed-component-version',
        'model-name', 
        'model-fullpath',
        'model-framework',#10
        'neo-compile']
    df = pd.DataFrame(None,
          columns=column_name,
          index=None)
    
    for d in device_list:
        print('Device "', d['thing-name'], '" has these ML components: ')
        component_list = get_installed_component_list(d['thing-name'])
        
        for c in component_list:
            config = get_config_info_as_yaml(config_file_path, c)
            if config == None:
                print('\t No ML components.')
                continue
            component_name, component_version = c
            print('\t component-name:', component_name, '-- version:', component_version)
            config = get_config_info_as_yaml(config_file_path, c)

            thing_group_name = d['thing-group-name']

            df = df.append({
                column_name[0]: config['deployment-name'],
                column_name[1]: config['target-group-arn'],
                column_name[2]: d['thing-name'],
                column_name[3]: d['status'],
                column_name[4]: d['last-status-updated'],
                column_name[5]: config['component-name'], 
                column_name[6]: config['component-version'], 
                column_name[7]: config['model-information']['original-model-path'].split('/')[-1],
                column_name[8]: config['model-information']['original-model-path'], 
                column_name[9]: config['model-information']['framework'], 
                column_name[10]: config['model-information']['compile-model']}, ignore_index=True)
            
    return df

以下のセルを実行すると、デプロイパイプラインを使って ML モデルをデプロイしたデバイスの一覧が Pandas の DataFrame 形式で表示されます。作成した Step Functions ワークフローを使わず直接 Greengrass のコンソールからコンポーネントをデプロイした場合は、こちらの一覧には表示されないのでご注意ください。実際のワークロードでは必ずワークフローを使ってデプロイすることをルールとすることをおすすめします。

`config_file_path_list` には、デプロイ設定ファイルが保存されている S3 パスのリストを設定します。

In [ ]:
config_file_path_list = [gg_s3_path]
info = None
for config_file_path in config_file_path_list:
    df = show_deployed_model_info(config_file_path)
    if info is None:
        info = df
    else:
        info = pd.concat([info, df])
info

[おまけ] DataFrame を html ファイルとして保存することもできます。

In [ ]:
pd.set_option('colheader_justify', 'center') 

html_string = '''
<html>
  <head><title>Pandas DataFrame</title></head>
  <link rel="stylesheet" type="text/css" href="dataframe-style.css"/>
  <body>
    {table}
  </body>
</html>.
'''

# OUTPUT AN HTML FILE
with open('index.html', 'w') as f:
    f.write(html_string.format(table=df.to_html(classes='table-style')))

Jupyter ノートブックで以下のセルを実行すると、html ファイルの中身が表示されます。Jupyter Lab や SageMaker Studio では `HTML` が機能しないので、以下のセルを実行するのではなく直接 html ファイルを開いてください。

In [ ]:
from IPython.display import HTML
HTML('index.html')

## [重要] リソースの削除


不要な課金を避けるために、以下のリソースを削除してください。特に Amazon SageMaker ノートブックインスタンスは削除しない限りコンピュートリソースとストレージの利用料金が継続するので、不要な場合は必ず削除してください。

- [必須] Amazon SageMaker ノートブックインスタンス
- 事前準備で作成した Cloud9 環境
- IoT Greengrass リソース
- S3 に保存したデータ
- AWS Step Functions ワークフロー
- Lambda 関数

### AWS Step Functions ワークフロー関連のリソース削除

以下のセルを実行して、作成したワークフローと IAM role, IAM policy を削除します。

In [ ]:
branching_workflow.delete()

def detach_role_policies(role_name):
    response = iam_client.list_attached_role_policies(
        RoleName=role_name,
    )
    policies = response['AttachedPolicies']

    for p in policies:
        response = iam_client.detach_role_policy(
            RoleName=role_name,
            PolicyArn=p['PolicyArn']
        )
    
detach_role_policies(step_functions_role_name)
iam_client.delete_role(RoleName=step_functions_role_name)
iam_client.delete_policy(PolicyArn=step_functions_policy_arn)

### Lambda 関数関連のリソース削除

以下のセルを実行して、作成した Lambda 関数と IAM role, IAM policy, ECR repository を削除します。

In [ ]:

for l in lambda_roles:
    detach_role_policies(l)
    iam_client.delete_role(RoleName=l)
    
for p in lambda_policies:
    iam_client.delete_policy(PolicyArn=p)
    
lambda_client.delete_function(FunctionName=lambda_function_name_update)
lambda_client.delete_function(FunctionName=lambda_function_name_compile)
lambda_client.delete_function(FunctionName=lambda_function_name_deploy)

ecr_client.delete_repository(
    repositoryName=ecr_repository_lambda_update,
    force=True
)
ecr_client.delete_repository(
    repositoryName=ecr_repository_lambda_compile,
    force=True
)
ecr_client.delete_repository(
    repositoryName=ecr_repository_lambda_deploy,
    force=True
)